In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# Copyright 2020 Confluent Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# A simple example demonstrating use of JSONSerializer.
import mysql.connector
import argparse
from uuid import uuid4
from six.moves import input
from confluent_kafka import Producer
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.json_schema import JSONSerializer
#from confluent_kafka.schema_registry import *
import pandas as pd
from typing import List

import time

API_KEY = 'AYMA6QVDYRIS6L3F'
ENDPOINT_SCHEMA_URL  = 'https://psrc-4nxg3.australia-southeast1.gcp.confluent.cloud'
API_SECRET_KEY = 'QSCStqXjRf78Ln8BIp87AgaCTcUiQ1EC0ByWUcurj40sSre6sCaPEH9INl/beo6o'
BOOTSTRAP_SERVER = 'pkc-xrnwx.asia-south2.gcp.confluent.cloud:9092'
SECURITY_PROTOCOL = 'SASL_SSL'
SSL_MACHENISM = 'PLAIN'
SCHEMA_REGISTRY_API_KEY = '3KA73SLQMTVU2DBW'
SCHEMA_REGISTRY_API_SECRET = '/t5erQbohU1E4YlG/W0b9NqoYaJppUYvzerPKIxCB43+yV58qPl7cpAOJsaG27j+'

col=['name_ran','created_at']

db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'root',
    'database': 'new'
}

def sasl_conf():

    sasl_conf = {'sasl.mechanism': SSL_MACHENISM,
                 # Set to SASL_SSL to enable TLS support.
                #  'security.protocol': 'SASL_PLAINTEXT'}
                'bootstrap.servers':BOOTSTRAP_SERVER,
                'security.protocol': SECURITY_PROTOCOL,
                'sasl.username': API_KEY,
                'sasl.password': API_SECRET_KEY
                }
    return sasl_conf



def schema_config():
    return {'url':ENDPOINT_SCHEMA_URL,
    
    'basic.auth.user.info':f"{SCHEMA_REGISTRY_API_KEY}:{SCHEMA_REGISTRY_API_SECRET}"

    }


class Ran:
    def __init__(self,record:dict):
        for k,v in record.items():
            v=str(v)
            setattr(self,k,v)
        self.record=record
    @staticmethod
    def dict_to_ran(data:dict,ctx):
        return Ran(record=data)

    def __str__(self):
        return f"{self.record}"
        
#reading data from sql

def reading_sql(conn):
    global last_created_at
    cursor = conn.cursor()
    sql="select * from kafka where created_at>(%s) order by created_at"
    last_created_at_query="select created_at from kafka where created_at > (%s) order by created_at desc limit 1"
    val=last_created_at
    
    cursor.execute(last_created_at_query,(val,))
#     print(cursor.fetchall())
    print(val)
    print(last_created_at)
    try:
        last_created_at=cursor.fetchall()[0][0]
    except:
        pass
    print(last_created_at)
    cursor.execute(sql,(val,))
    db=cursor.fetchall()
#     return "suc"
    
    
    for i in db:
        print(i)
        i=list(i)
        print(i[1])
        ran=Ran(dict(zip(col,i)))
        yield ran

def ran_to_dict(ran:Ran, ctx):
    """
    Returns a dict representation of a User instance for serialization.
    Args:
        user (User): User instance.
        ctx (SerializationContext): Metadata pertaining to the serialization
            operation.
    Returns:
        dict: Dict populated with user attributes to be serialized.
    """

    # User._address must not be serialized; omit from dict
    return ran.record


def delivery_report(err, msg):
    """
    Reports the success or failure of a message delivery.
    Args:
        err (KafkaError): The error that occurred on None on success.
        msg (Message): The message that was produced or failed.
    """

    if err is not None:
        print("Delivery failed for User record {}: {}".format(msg.key(), err))
        return
    print('User record {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))

def main(topic):
    schema_registry_conf = schema_config()
    schema_registry_client = SchemaRegistryClient(schema_registry_conf)
    subjects = schema_registry_client.get_subjects()
    print(subjects)
    subject = topic+'-value'

    schema = schema_registry_client.get_latest_version(subject)
    print("***")
    print(schema)
    print("***")
    schema_str=schema.schema.schema_str
    print(schema_str)

    string_serializer = StringSerializer('utf_8')
    json_serializer = JSONSerializer(schema_str, schema_registry_client, ran_to_dict)
    print(json_serializer)
    producer = Producer(sasl_conf())

    print("Producing user records to topic {}. ^C to exit.".format(topic))
    #while True:
        # Serve on_delivery callbacks from previous calls to produce()
    producer.poll(0.0)
    i=0
    
    while True:
        try:
            connection=mysql.connector.connect(**db_config)
            for ran in reading_sql(connection):
#                 if i<3:
                print("$%$%$%$")
                print(ran)
                print("*&*&&*&&*&")
                print(json_serializer(ran, SerializationContext(topic, MessageField.VALUE)))
                producer.produce(topic=topic,
                                key=string_serializer(str(uuid4())),
                                value=json_serializer(ran, SerializationContext(topic, MessageField.VALUE)),
                                on_delivery=delivery_report)
#                 else:
#                     break
#                 i+=1
        except KeyboardInterrupt:
            pass
        except ValueError:
            print("Invalid input, discarding record...")
            pass

        print("\nFlushing records...")
        producer.flush()
        time.sleep(5)

# main("car_topic")
last_created_at='' 
main("sql_ran")

['sql_ran-key', 'sql_ran-value', 'car_topic-value', 'car_topic-key']
***
***
{"$id":"http://example.com/myURI.schema.json","$schema":"http://json-schema.org/draft-07/schema#","additionalProperties":false,"description":"Sample schema to help you get started.","properties":{"created_at":{"description":"The type(v) type is used.","type":"string","format":"date-time"},"name_ran":{"description":"The type(v) type is used.","type":"string"}},"title":"SampleRecord","type":"object"}
Producing user records to topic sql_ran. ^C to exit.


2024-02-12 00:45:03.561761
('monu', '2024-02-07 00:22:23')
2024-02-07 00:22:23
$%$%$%$
{'name_ran': 'monu', 'created_at': '2024-02-07 00:22:23'}
*&*&&*&&*&
b'\x00\x00\x01\x86\xa4{"name_ran": "monu", "created_at": "2024-02-07 00:22:23"}'
('8f8d8fe6-854b-44f0-8483-8c0ed3a3d453', '2024-02-07 00:34:07')
2024-02-07 00:34:07
$%$%$%$
{'name_ran': '8f8d8fe6-854b-44f0-8483-8c0ed3a3d453', 'created_at': '2024-02-07 00:34:07'}
*&*&&*&&*&
b'\x00\x00\x01\x86\xa4{"name_ran": "

('5d1d671b-5a42-43de-a89f-f114aa745da5', '2024-02-07 16:29:37.500833')
2024-02-07 16:29:37.500833
$%$%$%$
{'name_ran': '5d1d671b-5a42-43de-a89f-f114aa745da5', 'created_at': '2024-02-07 16:29:37.500833'}
*&*&&*&&*&
b'\x00\x00\x01\x86\xa4{"name_ran": "5d1d671b-5a42-43de-a89f-f114aa745da5", "created_at": "2024-02-07 16:29:37.500833"}'
('97aa5cda-f783-491c-91d5-b659d9ef4707', '2024-02-07 16:29:42.534270')
2024-02-07 16:29:42.534270
$%$%$%$
{'name_ran': '97aa5cda-f783-491c-91d5-b659d9ef4707', 'created_at': '2024-02-07 16:29:42.534270'}
*&*&&*&&*&
b'\x00\x00\x01\x86\xa4{"name_ran": "97aa5cda-f783-491c-91d5-b659d9ef4707", "created_at": "2024-02-07 16:29:42.534270"}'
('aecf8099-0661-4456-bbd0-c5fe9e8b24af', '2024-02-07 16:29:47.560821')
2024-02-07 16:29:47.560821
$%$%$%$
{'name_ran': 'aecf8099-0661-4456-bbd0-c5fe9e8b24af', 'created_at': '2024-02-07 16:29:47.560821'}
*&*&&*&&*&
b'\x00\x00\x01\x86\xa4{"name_ran": "aecf8099-0661-4456-bbd0-c5fe9e8b24af", "created_at": "2024-02-07 16:29:47.560821"}

User record b'3717ba52-534d-4684-a68d-5ce8962c86ed' successfully produced to sql_ran [0] at offset 400
User record b'720c661d-0a91-4d4e-9d8f-5241be5cb02b' successfully produced to sql_ran [0] at offset 401
User record b'512ccb03-3bfa-4916-bfed-4a536ef5a30e' successfully produced to sql_ran [0] at offset 402
User record b'19957b32-0349-4748-96d2-123178c5c4f7' successfully produced to sql_ran [0] at offset 403
User record b'6a9ea0a7-d62f-49b4-af9a-68f97df2a719' successfully produced to sql_ran [0] at offset 404
User record b'56b3a1ac-7f47-4eb7-8482-7da6e8d4e69d' successfully produced to sql_ran [0] at offset 405
User record b'2d8e1895-8500-42b6-8224-8c091c5846d9' successfully produced to sql_ran [0] at offset 406
User record b'817ea968-37fc-4502-bf94-e297d5a70e9d' successfully produced to sql_ran [0] at offset 407
User record b'0a62041b-0878-4cca-9ece-d240735d6ce5' successfully produced to sql_ran [0] at offset 408
User record b'571bbe2c-1204-4295-a8a8-2625179c12fc' successfully produced

2024-02-12 00:45:03.561761
2024-02-12 00:45:03.561761
2024-02-12 00:45:08.586524
('61201daf-aba6-41ff-a155-c064c50edf29', '2024-02-12 00:45:08.586524')
2024-02-12 00:45:08.586524
$%$%$%$
{'name_ran': '61201daf-aba6-41ff-a155-c064c50edf29', 'created_at': '2024-02-12 00:45:08.586524'}
*&*&&*&&*&
b'\x00\x00\x01\x86\xa4{"name_ran": "61201daf-aba6-41ff-a155-c064c50edf29", "created_at": "2024-02-12 00:45:08.586524"}'

Flushing records...
User record b'7dc00cbf-b72c-4a50-8cb0-e0f0172a2f83' successfully produced to sql_ran [2] at offset 525
2024-02-12 00:45:08.586524
2024-02-12 00:45:08.586524
2024-02-12 00:45:13.600848
('d280e1d7-8805-4c7a-bcbf-fd5065f685d3', '2024-02-12 00:45:13.600848')
2024-02-12 00:45:13.600848
$%$%$%$
{'name_ran': 'd280e1d7-8805-4c7a-bcbf-fd5065f685d3', 'created_at': '2024-02-12 00:45:13.600848'}
*&*&&*&&*&
b'\x00\x00\x01\x86\xa4{"name_ran": "d280e1d7-8805-4c7a-bcbf-fd5065f685d3", "created_at": "2024-02-12 00:45:13.600848"}'

Flushing records...
User record b'71ce5c1c-84

2024-02-12 00:46:23.892574
2024-02-12 00:46:23.892574
2024-02-12 00:46:28.906988
('390da169-f7e3-413f-9e02-657f53909809', '2024-02-12 00:46:28.906988')
2024-02-12 00:46:28.906988
$%$%$%$
{'name_ran': '390da169-f7e3-413f-9e02-657f53909809', 'created_at': '2024-02-12 00:46:28.906988'}
*&*&&*&&*&
b'\x00\x00\x01\x86\xa4{"name_ran": "390da169-f7e3-413f-9e02-657f53909809", "created_at": "2024-02-12 00:46:28.906988"}'

Flushing records...
User record b'bab0aa94-926e-41e5-9551-8b43aaa1bece' successfully produced to sql_ran [2] at offset 534
2024-02-12 00:46:28.906988
2024-02-12 00:46:28.906988
2024-02-12 00:46:33.930315
('2a0d9d0b-5e15-43e0-9a40-61bdc5941f71', '2024-02-12 00:46:33.930315')
2024-02-12 00:46:33.930315
$%$%$%$
{'name_ran': '2a0d9d0b-5e15-43e0-9a40-61bdc5941f71', 'created_at': '2024-02-12 00:46:33.930315'}
*&*&&*&&*&
b'\x00\x00\x01\x86\xa4{"name_ran": "2a0d9d0b-5e15-43e0-9a40-61bdc5941f71", "created_at": "2024-02-12 00:46:33.930315"}'

Flushing records...
User record b'e32d3b0b-73